In [4]:
import pandas as pd
import datetime as dt
import os
import requests
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [5]:
orderbook = pd.read_csv('df_order_book_20200817.csv')
(orderbook.isnull().sum()/len(orderbook)) * 100

eventTime    0.0
bidPrice     0.0
bidQty       0.0
askPrice     0.0
askQty       0.0
depth        0.0
dtype: float64

In [ ]:
orderbook